In [1]:
import uuid
import random

from tqdm import tqdm
from threading import Thread
from threading import Thread, Lock

from cassandra.cluster import Cluster, BatchStatement
from cassandra.policies import RetryPolicy, ExponentialReconnectionPolicy

from utils.query_utils import *
from cassandra import ConsistencyLevel
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT

from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory
import time

In [2]:
profile = ExecutionProfile(
    retry_policy = DowngradingConsistencyRetryPolicy,
    consistency_level = ConsistencyLevel.TWO,
    serial_consistency_level = ConsistencyLevel.LOCAL_SERIAL,
    request_timeout = 200,
)

In [3]:
CLUSTER_IDS = ['172.21.0.2']
KEYSPACE = 'library_keyspace'

cluster = Cluster(CLUSTER_IDS, execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()
session.set_keyspace(KEYSPACE)

In [4]:
res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)
    reservation_id = r.reservation_id
    
reservation_id

d5124801-70de-459d-9185-0d014a6d5b8d db03f42d-32c1-4c35-8c86-d66019e0a591 User 2 Book 1798
be99ce1e-1c28-4160-aa98-26e2501b03ef db03f42d-32c1-4c35-8c86-d66019e0a591 User 2 Book 321
aeb0d5ed-cfc0-4569-bb85-657d9bd99b79 5312852e-49d1-4188-b68d-1729260e6482 User 87 Book 1658
8fe9c045-83b9-415b-ae97-99ee346b684e 8fe7bbdf-20b6-4f24-9bda-e6c7538e2e24 User 1 Book 273
c948e610-bc53-4f18-b1a7-b0d7128dc0d8 8fe7bbdf-20b6-4f24-9bda-e6c7538e2e24 User 1 Book 1177
ee05a9fd-52d2-48fe-8a52-77ff180bbec5 db03f42d-32c1-4c35-8c86-d66019e0a591 User 2 Book 1000
81921a4f-9ef7-4e5f-be8e-cb53866d4edc dd920adf-e59c-4960-a51c-73757625411d User 56 Book 414
4ce283e0-c146-4624-aba2-27074c815542 db03f42d-32c1-4c35-8c86-d66019e0a591 User 2 Book 1047
21a40477-fc28-4eaa-99fe-31b410bdb868 db03f42d-32c1-4c35-8c86-d66019e0a591 User 2 Book 739
c35bae35-7e11-46c2-a8d7-cb47290c3bd6 8fe7bbdf-20b6-4f24-9bda-e6c7538e2e24 User 1 Book 802
beb72bfb-d41e-4579-980c-57bec73095ff a9afa4a2-735c-402b-925e-20158de3a0ac User 10 Book 979
54

UUID('e8228912-65fa-4332-a25a-5663b62d3501')

In [5]:
res = list(get_all_books(session))
book_id, book_name = None, None
print(len(res))
for r in res:
    print(r.book_id, r.book_name, r.is_reserved)
    # if r.is_reserved:
    #     print(r.book_id, r.book_name, r.is_reserved)
    if r.book_name == "Book 1":
        book_id = r.book_id
        book_name = "Book 1"

book_id, book_name

2001
fda46e35-f2c1-4dba-9c5b-da318ee6bd1d Book 236 True
a7e06166-e89a-474e-9d35-87c3146e7ab9 Book 840 True
546d0c81-6d5d-44c4-aecc-a8c4a600662a Book 1627 True
09031c12-bd87-4d74-90e8-a1351cf0b287 Book 622 True
b0a1191b-cb67-4463-8f36-2a60b4e53c59 Book 1469 True
5e16858d-4fc9-4e26-ad23-3363c4466280 Book 1077 True
580c8fcf-cd3f-4c82-858a-7ac64b00c866 Book 74 True
b8c44a83-cbe9-4284-9f8b-9a5619c7b1df Book 177 True
ae06f762-d3d0-4360-a4a2-19b70591e9b1 Book 1718 True
82026c4b-e730-4d7b-b66d-9b14b773a9e8 Book 345 True
a7819bb6-cd3c-4a2e-b86f-7a2f6aac2fd1 Book 621 True
c1827b32-dab6-4d7c-8389-aaf632c57310 Book 427 True
1152bac7-4732-4bc5-86e9-101ed9871463 Book 296 True
4ca25164-db14-4223-a38d-df1043f92c13 Book 1943 True
a0b15b91-6e94-4329-aa2c-9d84dddc0235 Book 819 True
e80bf6ac-ccf2-49f8-bcaf-e96d9568ed5f Book 1266 True
d49819ff-a0f4-41f0-9ec8-187af82c954b Book 1999 True
98550dc3-9e0f-4f1e-8570-eed9447f8c67 Book 820 True
20bb3aa8-ac84-4768-b1fe-bfcebd4fc88f Book 96 True
3497a49e-0a90-4785-b4

(UUID('eefe276b-4f77-4ae8-9da9-6809b033647d'), 'Book 1')

In [6]:
res = list(get_all_users(session))
for r in res:
    if r.user_name == "User 1":
        user_name = "User 1"
        user_id = r.user_id
    # if r.reservation_ids_list is not None:
    #     print(r.user_id, r.user_name, r.reservation_ids_list)
    print(r.user_id, r.user_name, r.reservation_ids_list)

user_name, user_id

afd86705-e47d-46bf-b1cc-397af0438910 User 55 [UUID('ef5679f3-7d99-453a-9896-72044773f2b8'), UUID('af4e3ce9-6837-4ddf-b181-2afc240d33b1'), UUID('faabec19-cc78-400e-b939-80ee64aa1058'), UUID('def39707-0215-4765-82cd-9402beebba47'), UUID('b16da0c5-594e-41d8-97f2-c25f8e80789b'), UUID('db84e14e-421b-4e9a-80df-fd77f2a45688'), UUID('9c96bc9e-5c97-42bf-b45e-19fb4ceddc34'), UUID('f8ead901-7e31-43ba-9659-71e472c7d59b')]
77818418-540d-46e9-9a4c-1aca44ab1c7b User 20 [UUID('2d49182c-9f8a-487f-ba81-d275f46b25ca'), UUID('e5b49ee7-bc58-4504-a2c0-4f7ce75ad290'), UUID('008f4558-150f-4c00-8d4a-54c586c1d38f'), UUID('65898ad2-a504-4cf7-8fe7-918989dd60f1'), UUID('69ee9d0a-5784-4a95-9be0-e8fde36f924f'), UUID('35db221c-b283-42df-94f4-2903733e9906'), UUID('2410f39b-7d01-40f3-ab46-e718319b6c8c'), UUID('6125f788-2c80-4515-a2d5-bf37c6796447'), UUID('97b06afc-32a2-4d0d-8fda-1e66d61988d8')]
fc6bc749-c581-4cdc-8d8a-754352b09f0c User 13 [UUID('c3cdf387-f4a2-4330-96a4-8af13966a7da'), UUID('b6ac2c75-6fa7-4d79-998e-6923

('User 1', UUID('8fe7bbdf-20b6-4f24-9bda-e6c7538e2e24'))

In [7]:
def test_reserve_cancel_debug(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    # for _ in range(repeat_num):
    for _ in range(3):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)

        print("Created")
        res = get_all_reservations(session)
        for r in res:
            if r.reservation_id == reservation_id:
                print(r.reservation_id, r.user_id, r.user_name, r.book_name)

        cancel_reservation(session, reservation_id)
        print("Cancelled")
        res = get_all_reservations(session)
        for r in res:
            if r.reservation_id == reservation_id:
                print(r.reservation_id, r.user_id, r.user_name, r.book_name)


def test_update_reservations_debug(session, num_reservations=1000):
    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        assert reservation.book_id != new_book.book_id
        assert reservation.book_name != new_book.book_name

        update_reservation_book(session, reservation.reservation_id, new_book.book_id)
        
        updated_reservation = get_reservation_by_id(session, reservation.reservation_id)
        
        assert updated_reservation.book_id == new_book.book_id
        assert updated_reservation.book_name == new_book.book_name

def test_make_random_requests_two_users_debug(user1, user2, num_repeats=10_000):
    # Simulates two users making 10_000 total random requests

    def make_random_requests(user_id, user_name, num_actions):
        # Simulates a user making random requests num_actions number of times

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        possible_requests = ["update_reservation_book", "update_reservation_user", "cancel_reservation", "make_reservation"]

        while num_actions > 0:

            selected_action = random.choice(possible_requests)

            # Update DB state information
            reservations = list(get_all_reservations(user_session))

            # If no reservations left, make a reservation
            if len(reservations) == 0:
                selected_action = "make_reservation"

            books = list(get_all_books(user_session))
            users = list(get_all_users(user_session))

            if selected_action == "update_reservation_book":
                reservation = random.choice(reservations)
                book = random.choice(books)

                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")
                    
                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("   Book:", book)
                print("============================")
                    
                # Does not assume book is available
                update_reservation_book(user_session, reservation.reservation_id, book.book_id)

            elif selected_action == "update_reservation_user":
                reservation = random.choice(reservations)
                user = random.choice(users)
                
                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("   User:", user)
                print("============================")

                # Does not assume user does not possess reservation already
                update_reservation_user(user_session, reservation.reservation_id, user.user_id)

            elif selected_action == "cancel_reservation":
                reservation = random.choice(reservations)
                
                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")
                
                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("============================")

                cancel_reservation(user_session, reservation.reservation_id)

            elif selected_action == "make_reservation":
                available_books = [book for book in books if not book.is_reserved]

                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Available books:", len(available_books))

                # There are books available
                if len(available_books) > 0:
                    selected_random_book = random.choice(available_books)
                    print("   Selected book:", selected_random_book)

                    try:
                        add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    except:
                        pass
                    
                # No available books, dont use action
                else:
                    num_actions += 1

                print("============================")

            num_actions -= 1

        user_session.shutdown()

    thread1 = Thread(target=make_random_requests, args=[user1.user_id, user1.user_name, num_repeats//2])
    thread2 = Thread(target=make_random_requests, args=[user2.user_id, user2.user_name, num_repeats//2])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

In [8]:
def test_same_request(session, reservation_id, new_book_id, repeat_num=10_000):
    # Runs an update of the book in a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        update_reservation_book(session, reservation_id, new_book_id)


def test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)


def test_update_reservations(session, num_reservations=1000):
    # Updates a 1000 reservations

    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        update_reservation_book(session, reservation.reservation_id, new_book.book_id)

lock = Lock()

# TODO: currently the same book can be reserved by two users.
# Add random wait time before checking availability/locks/idk
def test_make_all_reservations_two_users(user1, user2):
    # Simulates two users reserving all possible books

    def make_all_reservations(user_id, user_name):
        # Simulates user making all possible reservations one by one

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        # Update available book state
        books = get_all_books(user_session)
        available_books = [book for book in books if not book.is_reserved]
    
        while len(available_books) > 0:
            selected_random_book = random.choice(available_books)
            
            try:
                with lock:
                    add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    time.sleep(random.uniform(0.01, 0.1))
            except:
                pass
            
            with lock:
                books = get_all_books(user_session)
                available_books = [book for book in books if not book.is_reserved]

        user_session.shutdown()

    thread1 = Thread(target=make_all_reservations, args=[user1.user_id, user1.user_name])
    thread2 = Thread(target=make_all_reservations, args=[user2.user_id, user2.user_name])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

def test_make_random_requests_two_users(user1, user2, num_repeats=10_000):
    # Simulates two users making 10_000 total random requests

    def make_random_requests(user_id, user_name, num_actions):
        # Simulates a user making random requests num_actions number of times

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        possible_requests = ["update_reservation_book", "update_reservation_user", "cancel_reservation", "make_reservation"]

        while num_actions > 0:

            selected_action = random.choice(possible_requests)

            # Update DB state information
            reservations = list(get_all_reservations(user_session))

            # If no reservations left, make a reservation
            if len(reservations) == 0:
                selected_action = "make_reservation"

            books = list(get_all_books(user_session))
            users = list(get_all_users(user_session))
            
            if selected_action == "update_reservation_book":
                reservation = random.choice(reservations)
                book = random.choice(books)

                # Does not assume book is available
                update_reservation_book(user_session, reservation.reservation_id, book.book_id)

            elif selected_action == "update_reservation_user":
                reservation = random.choice(reservations)
                user = random.choice(users)

                # Does not assume user does not possess reservation already
                update_reservation_user(user_session, reservation.reservation_id, user.user_id)

            elif selected_action == "cancel_reservation":
                reservation = random.choice(reservations)
                cancel_reservation(user_session, reservation.reservation_id)

            elif selected_action == "make_reservation":
                available_books = [book for book in books if not book.is_reserved]

                # There are books available
                if len(available_books) > 0:
                    selected_random_book = random.choice(available_books)

                    try:
                        add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    except:
                        pass
                    
                # No available books, dont use action
                else:
                    num_actions += 1

            num_actions -= 1

        user_session.shutdown()

    thread1 = Thread(target=make_random_requests, args=[user1.user_id, user1.user_name, num_repeats//2])
    thread2 = Thread(target=make_random_requests, args=[user2.user_id, user2.user_name, num_repeats//2])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

In [9]:
users = get_all_users(session)
user1, user2 = None, None
for user in users:
    if user.user_name == "User 1":
        user1 = user

    if user.user_name == "User 2":
        user2 = user

user1, user2

(Row(user_id=UUID('8fe7bbdf-20b6-4f24-9bda-e6c7538e2e24'), reservation_ids_list=[UUID('8ec8f73c-56fb-4e5f-b9a4-ffbcebab311a'), UUID('538310bf-deab-4be6-84b6-d8de33750f77'), UUID('1747b2a9-aa18-4cb8-ba70-b782a43a8910'), UUID('b489c643-8e5c-456d-b7c3-3624b29a85a8'), UUID('5071cfea-beb4-4e16-a15a-9b9d4cdc4b32'), UUID('12366e78-dbaa-4142-ab1f-6dd100071b39'), UUID('3106d57e-a35b-4031-9090-360d62e1ca38'), UUID('829f2358-be0c-4e3a-94c6-a9b6226910c0'), UUID('2ef64d81-2159-4585-b7e8-987f8d60172e'), UUID('a32b1c72-265e-43ea-90b3-b305460e2571'), UUID('86c02d5d-d818-4437-a7e6-0c8ca991b493'), UUID('9e2bb68a-71d8-4008-9bba-c6ff7df18822'), UUID('397dab11-adbc-48fc-97b1-919629d406eb'), UUID('027cbd7f-4977-426e-a6e4-cb5a1af2b451'), UUID('9f7a209c-d7e2-45d9-a690-582eae1a101c'), UUID('7c5be24c-ae6a-4735-b711-b733fa856a09'), UUID('f193eb8c-73d8-478c-82ec-5d6068af3a3d'), UUID('73406420-980d-4710-95ed-03e54a53ad47'), UUID('a011b73b-d6e0-4a64-be91-2a7277509c8c'), UUID('aba59ef0-d0ae-423f-9df7-109860bb3ac4'),

In [10]:
test_make_random_requests_two_users(user1, user2, 10_000)

Error occurred while updating book reservation status: Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error canceling the reservation:  Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error occurred while updating book reservation status: Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error canceling the reservation:  Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"


In [11]:
test_make_all_reservations_two_users(user1, user2)

In [12]:
all_reservations = get_all_reservations(session)

user_1_reservations = 0
user_2_reservations = 0
for reservation in all_reservations:
    if reservation.user_id == user1.user_id:
        user_1_reservations += 1

    if reservation.user_id == user2.user_id:
        user_2_reservations += 1

print(f"User 1 reservation count: {user_1_reservations}")
print(f"User 2 reservation count: {user_2_reservations}")

User 1 reservation count: 533
User 2 reservation count: 572


In [13]:
res = list(get_all_reservations(session))
user_1_reservations = [r for r in res if r.user_id == user1.user_id]
user_2_reservations = [r for r in res if r.user_id == user2.user_id]

user_1 = get_user(session, user1.user_id)
user_2 = get_user(session, user2.user_id)

for r in user_1_reservations:
    if r in user_2_reservations:
        print("Error user1 reservation in user2 reservations list")

    if r.reservation_id not in user_1.reservation_ids_list:
        print("Error user1 reservation not in list")

for r in user_2_reservations:
    if r in user_1_reservations:
        print("Error user2 reservation in user1 reservations list")

    if r.reservation_id not in user_2.reservation_ids_list:
        print("Error user2 reservation not in list")

In [15]:
book_ids_1 = set([r.book_id for r in user_1_reservations])
book_ids_2 = set([r.book_id for r in user_2_reservations])

assert len(book_ids_1 & book_ids_2) == 0, "No book should be reserved by both clients at the same time"

AssertionError: No book should be reserved by both clients at the same time

In [16]:
book_ids_1 & book_ids_2

{UUID('1344d167-6e39-440a-829e-40dd00906f11'),
 UUID('1680b7c6-4a51-47cf-949f-157afa434dcf'),
 UUID('5d21a7dc-1647-49e3-abb6-b2d9f39fdf0d'),
 UUID('6770d7c0-d416-43ab-8e9d-6f2ee4110f85'),
 UUID('a4cdf58a-2924-4147-b89b-865a61035b4d'),
 UUID('b00a16e6-6fde-483e-a112-1f78801b88cd'),
 UUID('c42a2418-fae7-41f4-8b0e-88e85f74cd4c'),
 UUID('d472ee6b-f4bc-4fbc-a6bf-8bcf42fd2c22')}

In [ ]:
book = list(book_ids_1 & book_ids_2)[0]
for res in get_all_reservations(session):
    if res.book_id == book:
        print(res.reservation_id, res.user_name, res.book_name, res.book_id)

IndexError: list index out of range

In [ ]:
books = get_all_books(session)
available_books = [book for book in books if not book.is_reserved]
available_books

[]

In [ ]:
test_update_reservations(session)

AssertionError: There should be at least a 1000 available books

In [ ]:
test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name)

res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)
    

100%|██████████| 10000/10000 [03:33<00:00, 46.90it/s]


44c4f1be-d266-437b-9dcf-bbfe45eec299 a26c83c2-3a1b-4f26-b18c-a0a8385bc492 User 1 Book 1001


In [ ]:
test_same_request(session, reservation_id, book_id)

res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)

100%|██████████| 10000/10000 [00:44<00:00, 226.46it/s]

44c4f1be-d266-437b-9dcf-bbfe45eec299 a26c83c2-3a1b-4f26-b18c-a0a8385bc492 User 1 Book 1


In [ ]:
session.shutdown()
cluster.shutdown()